In [28]:
def list_blobs(bucket_name,pdf_path):
    """Lists all the blobs in the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    blobs = bucket.list_blobs(prefix="Anthem A")
    
    for blob in blobs:
        path="{}/{}".format(bucket_name,blob.name)
        pdf_path.append(path)
        #print path
    return pdf_path   

In [29]:
from google.cloud import storage
bucket="sampleeob"
pdf_path=[]
list_blobs(bucket,pdf_path)
#print pdf_path

['sampleeob/Anthem A/',
 'sampleeob/Anthem A/processed_Anthem.1.pdf',
 'sampleeob/Anthem A/processed_Anthem.10.pdf',
 'sampleeob/Anthem A/processed_Anthem.2.pdf',
 'sampleeob/Anthem A/processed_Anthem.3.pdf',
 'sampleeob/Anthem A/processed_Anthem.4.pdf',
 'sampleeob/Anthem A/processed_Anthem.6.pdf',
 'sampleeob/Anthem A/processed_Anthem.7.pdf',
 'sampleeob/Anthem A/processed_Anthem.9.pdf']

In [30]:
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

from pdf2image import convert_from_path
import os

options = PipelineOptions()

p = beam.Pipeline(options=options)

In [31]:
class Download_PDF(beam.DoFn):    
    def process(self,pdf_path):
        print 1
        """Downloads a blob from the bucket."""
        bucket_name=pdf_path.split("/")[0]
        source_blob=pdf_path.split("/",1)[-1]
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(source_blob)
        if source_blob.endswith(".pdf"):
            blob.download_to_filename("{}/{}".format(bucket_name,source_blob))
        print("{}/{}".format(bucket_name,blob.name))    
        return ["{}/{}".format(bucket_name,blob.name)]
    


In [32]:
class Convert_PDF_to_JPG(beam.DoFn):
    def process(self,path):
        print 2
        print path
        #pdf_dir= rootdir + "/" + d
        #path="{}".format(pdf_dir)
        if path.endswith(".pdf"):
                        pages=convert_from_path(path,300) 
                        bucket_name=path.split("/")[0]
                        destination="unlabelled/{}".format(path.split("/")[-1])
                        #pdf_file gives name of pdf file
                        file_name=path.split("/")[-1]
                        pdf_file=file_name[:-4]
                        #make a directory JPEGs->pdf_file to save images of all pages of the particular pdf file
                        if not os.path.exists("{}/{}".format("sampleeob/JPEGs",pdf_file)):
                            os.mkdir("{}/{}".format("sampleeob/JPEGs",pdf_file))
                        images_path1="{}/{}".format("sampleeob/JPEGs",pdf_file)
                        for page in pages:
                            page.save("{}/{}-page{}.jpg".format(images_path1,pdf_file,pages.index(page)),"JPEG")
                            page_name="{}-page{}.jpg".format(pdf_file,pages.index(page))
                        files=os.listdir("{}".format(images_path1))
                        files=[images_path1+"/"+ f for f in files]
                        print files
                        return files
                        #for f in files:
                            #Function to upload all the images of pdf file in the bucket
                            #upload_blob(bucket1,"{}/{}/{}".format("JPEGs",pdf_file,f),  "{}/{}/{}".format(dest_folder,pdf_file,f))
                        #subprocess.call("gsutil -m cp -r {} {}".format(path,destination), shell=True)
                        #logging.info("{},{},converted".format(timestamp,pdf_dir))
                        #path1="{}/{}".format(file1,d)
                        #idx=path1.rfind("/")
                        #Adding processed keyword before pdf_file name 
                        #path2=path1[:idx+1]+"processed_"+path1[idx+1:]
                        #Renaming function to change pdf_file name to processed_pdf_file name
                        #rename_blob(bucket1,"{}/{}".format(file1,d),"{}".format(path2))

In [33]:
class Upload_Images_to_Bucket(beam.DoFn):
    def process(self,path):
        print 3
        print path
        """Uploads a file to the bucket."""
        bucket_name=path.split("/")[0]
        source_file_name=path
        destination_blob_name="unlabelled/{}".format(path.split("/",2)[-1])
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_filename(source_file_name)
        

In [34]:
with beam.Pipeline() as p:
    lines = (p |beam.Create(pdf_path))
    download_file_path = lines |  beam.ParDo(Download_PDF()) 
    download_file_path | 'Write 1' >> beam.io.WriteToText('sampleeob/filename.txt')
    images_path=download_file_path | beam.ParDo(Convert_PDF_to_JPG())
    images_path | 'Write 2' >> beam.io.WriteToText('sampleeob/filename.txt')
    images_path | beam.ParDo(Upload_Images_to_Bucket())
    result=p.run()
#images_path=download_file_path | beam.ParDo(Convert_PDF_to_JPG())
#status = images_path | beam.ParDo(Upload_Images_to_Bucket())

1
sampleeob/Anthem A/
2
sampleeob/Anthem A/
1
sampleeob/Anthem A/processed_Anthem.1.pdf
2
sampleeob/Anthem A/processed_Anthem.1.pdf
['sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page15.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page8.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page18.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page10.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page14.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page3.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page6.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page21.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page7.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page11.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page16.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page12.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-

1
sampleeob/Anthem A/
2
sampleeob/Anthem A/
1
sampleeob/Anthem A/processed_Anthem.1.pdf
2
sampleeob/Anthem A/processed_Anthem.1.pdf
['sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page15.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page8.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page18.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page10.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page14.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page3.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page6.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page21.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page7.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page11.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page16.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-page12.jpg', 'sampleeob/JPEGs/processed_Anthem.1/processed_Anthem.1-